#### <h1> Module 4A </h1>


In [4]:
from cellorganizer.tools import slml2img, __does_file_exist
import datetime
import glob
import urllib.request
import hashlib
import os
from IPython.core.display import display, HTML, Image
from IPython.display import Code, FileLink

module_name = 'Module4A'
models_dir = '/home/murphylab/cellorganizer/local/models/'

###########################################################################
# FEEL FREE TO MODIFY THE VARIABLES IN THIS BLOCK
# CellOrganizer model path relative to `model_path`
model_name = os.path.join('3D', 'SPHARM-RPDM', 'HeLa_3D_SPHARM_lamp2.mat')
# Use `'framework'` for cell and nucleus, `'all'` to also create objects
# for other organelle models in the CellOranizer model
synthesis = 'framework'
# Number of geometries to produce
numberOfSynthesizedImages = 5
vcml_name = 'SarmaGhosh2012ForJupyter.vcml'
# Set to `None` if you copied the VCML file `vcml_name` to `models_dir`
vcml_gist_url = 'https://gist.github.com/tebuck/535527cbc41047a51a49830b779bbe45/archive/0adfceab6d114e686bfbab09510c1e5169c47371.zip'
vcml_gist_sha256_expected = '34c822d52adf72909206f599e9ca2d9cdf78f6532e7845c16dbfa6a7a0695300'
# Simulation time and accuracy parameters
default_time_step = 1e0
end_time = 1000
output_time_step = 100
max_time_step = 4
absolute_tolerance = 1e-8
relative_tolerance = 1e-8
###########################################################################

model_path = os.path.join(models_dir, model_name)
vcml_models_dir = os.path.join(models_dir, 'VCML')
vcml_path = os.path.join(vcml_models_dir, vcml_name)

def get_reaction_network_file():
    print('Attempting to retrieve {}'.format(vcml_gist_url))
    if __does_file_exist(vcml_gist_url):
        print( 'Creating directory {}'.format(vcml_models_dir) )
        os.makedirs(vcml_models_dir, exist_ok=True)
        print('Saving as {}'.format(vcml_path))
        urllib.request.urlretrieve( vcml_gist_url, vcml_path + '.zip' )
        with open(vcml_path + '.zip', 'rb') as f:
            vcml_gist_sha256 = hashlib.sha256(f.read()).hexdigest()
        assert vcml_gist_sha256 == vcml_gist_sha256_expected, 'Retrieved file is not the same as expected, hash differs'
        os.system( 'unzip -jod {} {}'.format(vcml_models_dir, vcml_path + '.zip'))
        os.system( 'rm -fv {}'.format(vcml_path + '.zip') )
    else:
        print('Unable to find file. Check later.')
        raise(FileNotFoundError)

if vcml_gist_url is not None:
    get_reaction_network_file()

topdir = "/home/murphylab/cellorganizer/local/results"
outputdir = os.path.join(topdir, module_name)
os.makedirs(topdir, exist_ok=True)
os.chdir(topdir)
os.system("ls")
os.makedirs(outputdir, exist_ok=True)
os.chdir(outputdir)
os.system("ls")

# Set options

options = {}

# Set general options

# Random seed. Change to produce a different sample of geometries.
options['seed'] = 691357
options['synthesis'] = synthesis
options['targetDirectory'] = outputdir
options['prefix'] = module_name
# options['output_filename'] = module_name 
options['verbose'] = True
options['debug'] = False
options['display'] = False
options['model.spharm_rpdm.synthesis_method'] = 'random_sampling'
options['model.spharm_rpdm.imageSize'] = [205, 205, 18]
options['numberOfSynthesizedImages'] = numberOfSynthesizedImages
options['rendAtStd'] = 1;
options['overlapthresh'] = 1
options['compression'] = 'lzw'
options['microscope'] = 'none'
options['sampling.method'] = 'disc'
# Output types
options['output.tifimages'] = True
options['output.shape_space_coords'] = True
options['output.OMETIFF'] = True
options['output.indexedimage'] = True
options['output.indexedimage'] = True
downsampling = 1/2
if options['synthesis'] == 'all':
    downsampling *= 1/4;
options['output.VCML.writeVCML'] = True
options['VCML.downsampling'] = downsampling
options['oobbuffer'] = 0.1
options['VCML.input_filename'] = vcml_path
options['NET.useImageAdjacency'] = False
options['VCML.defaultTimeStep'] = default_time_step
options['VCML.endTime'] = end_time
options['VCML.outputTimeStep'] = output_time_step
options['VCML.maxTimeStep'] = max_time_step
options['VCML.absoluteTolerance'] = absolute_tolerance
options['VCML.relativeTolerance'] = relative_tolerance

answer = slml2img([model_path], options)

# Print paths of generated files
print('Generated VCML files:')
for output_vcml_path in sorted(glob.glob(os.path.join(outputdir, options['prefix'], 'cell*', 'cell.vcml'))):
    print(output_vcml_path)
    
# Display visualizations of generated images
# for image_path in glob.glob(os.path.join(outputdir, options['prefix'], 'cell*', 'indexed.png')):
    # display(Image(filename=image_path))


Attempting to retrieve https://gist.github.com/tebuck/535527cbc41047a51a49830b779bbe45/archive/0adfceab6d114e686bfbab09510c1e5169c47371.zip
Creating directory /home/murphylab/cellorganizer/local/models/VCML
Saving as /home/murphylab/cellorganizer/local/models/VCML/SarmaGhosh2012ForJupyter.vcml
Generated VCML files:
/home/murphylab/cellorganizer/local/results/Module4A/Module4A/cell1/cell.vcml
/home/murphylab/cellorganizer/local/results/Module4A/Module4A/cell2/cell.vcml
/home/murphylab/cellorganizer/local/results/Module4A/Module4A/cell3/cell.vcml
/home/murphylab/cellorganizer/local/results/Module4A/Module4A/cell4/cell.vcml
/home/murphylab/cellorganizer/local/results/Module4A/Module4A/cell5/cell.vcml
